# CSCI 3202, Spring 2025
# Final Project
# 100 Points
# Due: March 21 at 11:59 pm

<br> 

### Your name:Tanmay Meti and Justin Goh

<br> 

---

## Mancala rules for this homework assignment
**(there are many different rules sets for Mancala.  Please read this before writing the code)**

* Players sit on opposite sides of the long edge of the board
* There are 6 small pits in the middle of the board and 2 large ones at each end.  The small ones in the middle and the large pit on your right are yours.  The small ones on the other side and the large pit to your opponent's right are theirs
* The large pits at the end of the board are called Mancalas
* Set up the board with 4 stones per small pit (none in the mancalas)
* On every turn, select a pit on your side of the board that contains one or more stones,  then distribute its stones, one stone per pit, in an counter-clockwise direction until you have no stones remaining
* If you encounter your opponent's mandala, skip it
* If you encounter your mancala, drop a stone into it
* If the last stone lands in an empty pit on your side of the board, capture this stone and any stones in your opponent's pit on the other side of the board, collect all of these stones, including the one that just landed, and place them into your mancala.
* If either player's pits are entirely empty, the game concludes. 
* The player who still has stones on his side of the board when the game concludes places all of these pieces into their mancala.
The player with the most stones in their mancala is declared the winner. If both players have an equal number of stones in their mancala, the game results in a tie.


In [1]:
class Mancala:
    def __init__(self, pits_per_player=6, stones_per_pit = 4):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones 
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board)-1-1]
        self.p2_mancala_index = len(self.board)-1
        
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

    def display_board(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)
        
    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        """
        pit -= 1
        if self.current_player == 1:
            if 0 <= pit <= self.p1_pits_index[1] and self.board[pit] > 0:
                return True
        elif self.current_player == 2:
            real_index = pit + self.p1_mancala_index + 1
            if self.p2_pits_index[0] <= real_index <= self.p2_pits_index[1] and self.board[real_index] > 0:
                return True
        return False
        pass
        
    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """
        valid_pits = []
        if self.current_player == 1:
            for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1):
                if self.board[i] > 0:
                    valid_pits.append(i + 1)
        else:
            for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1):
                if self.board[i] > 0:
                    valid_pits.append(i - self.p1_mancala_index)
        return random.choice(valid_pits) if valid_pits else None
        pass
    
    def play(self, pit):
        """
        This function simulates a single move made by a specific player using their selected pit. It primarily performs three tasks:
        1. It checks if the chosen pit is a valid move for the current player. If not, it prints "INVALID MOVE" and takes no action.
        2. It verifies if the game board has already reached a winning state. If so, it prints "GAME OVER" and takes no further action.
        3. After passing the above two checks, it proceeds to distribute the stones according to the specified Mancala rules.

        Finally, the function then switches the current player, allowing the other player to take their turn.
        """
        if self.winning_eval():
            print("GAME OVER")
            return self.board
        if not self.valid_move(pit):
            print("INVALID MOVE")
            return self.board
        pit -= 1 
        if self.current_player == 1:
            index = pit
            mancala = self.p1_mancala_index
            opponent_mancala = self.p2_mancala_index
        else:
            index = pit + self.p1_mancala_index + 1
            mancala = self.p2_mancala_index
            opponent_mancala = self.p1_mancala_index
        stones = self.board[index]
        self.board[index] = 0
        current_index = index
        while stones > 0:
            current_index = (current_index + 1) % len(self.board)
            if current_index == opponent_mancala:
                continue
            self.board[current_index] += 1
            stones -= 1
        if self.current_player == 1 and 0 <= current_index <= self.p1_pits_index[1] and self.board[current_index] == 1:
            opposite_index = self.p2_pits_index[1] - (current_index - self.p1_pits_index[0])
            captured = self.board[opposite_index]
            if captured > 0:
                self.board[self.p1_mancala_index] += captured + 1
                self.board[opposite_index] = 0
                self.board[current_index] = 0
        elif self.current_player == 2 and self.p2_pits_index[0] <= current_index <= self.p2_pits_index[1] and self.board[current_index] == 1:
            opposite_index = self.p1_pits_index[1] - (current_index - self.p2_pits_index[0])
            captured = self.board[opposite_index]
            if captured > 0:
                self.board[self.p2_mancala_index] += captured + 1
                self.board[opposite_index] = 0
                self.board[current_index] = 0
        if (self.current_player == 1 and current_index == self.p1_mancala_index) or (self.current_player == 2 and current_index == self.p2_mancala_index):
            pass  
        else:
            self.current_player = 2 if self.current_player == 1 else 1
        self.moves.append((self.current_player, pit + 1))
        return self.board
    def winning_eval(self):
        """
        Function to verify if the game board has reached the winning state.
        Hint: If either of the players' pits are all empty, then it is considered a winning state.
        """
        p1_empty = all(self.board[i] == 0 for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1))
        p2_empty = all(self.board[i] == 0 for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1))
        if p1_empty or p2_empty:
            self.board[self.p1_mancala_index] += sum(self.board[self.p1_pits_index[0]:self.p1_pits_index[1] + 1])
            self.board[self.p2_mancala_index] += sum(self.board[self.p2_pits_index[0]:self.p2_pits_index[1] + 1])
            for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1):
                self.board[i] = 0
            for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1):
                self.board[i] = 0
            if self.board[self.p1_mancala_index] > self.board[self.p2_mancala_index]:
                print("Player 1 wins!")
            elif self.board[self.p1_mancala_index] < self.board[self.p2_mancala_index]:
                print("Player 2 wins!")
            else:
                print("It's a tie!")
            return True
        return False

In [2]:
# Mancala part 1 
game = Mancala()
game.display_board()

# Player 1 selects pit 1 (1-based index)
game.play(1)
game.display_board()

# Player 2 selects pit 2
game.play(2)
game.display_board()

# Player 1 selects pit 3
game.play(3)
game.display_board()

# Player 2 selects pit 2
game.play(2)
game.display_board()

# Player 1 selects pit 1
game.play(1)
game.display_board()

# Printing the list of moves
print("\nList of valid moves:")
for move in game.moves:
    player, pit = move
    print(f"Player {player} selected pit {pit}")

P1               P2
     ____0____     
1 -> | 4 | 4 | <- 6
2 -> | 4 | 4 | <- 5
3 -> | 4 | 4 | <- 4
4 -> | 4 | 4 | <- 3
5 -> | 4 | 4 | <- 2
6 -> |_4_|_4_| <- 1
         0         
Turn: P1
P1               P2
     ____0____     
1 -> | 0 | 4 | <- 6
2 -> | 5 | 4 | <- 5
3 -> | 5 | 4 | <- 4
4 -> | 5 | 4 | <- 3
5 -> | 5 | 4 | <- 2
6 -> |_4_|_4_| <- 1
         0         
Turn: P2
P1               P2
     ____0____     
1 -> | 0 | 5 | <- 6
2 -> | 5 | 5 | <- 5
3 -> | 5 | 5 | <- 4
4 -> | 5 | 5 | <- 3
5 -> | 5 | 0 | <- 2
6 -> |_4_|_4_| <- 1
         0         
Turn: P1
P1               P2
     ____0____     
1 -> | 0 | 5 | <- 6
2 -> | 5 | 5 | <- 5
3 -> | 0 | 5 | <- 4
4 -> | 6 | 5 | <- 3
5 -> | 6 | 0 | <- 2
6 -> |_5_|_5_| <- 1
         1         
Turn: P2
INVALID MOVE
P1               P2
     ____0____     
1 -> | 0 | 5 | <- 6
2 -> | 5 | 5 | <- 5
3 -> | 0 | 5 | <- 4
4 -> | 6 | 5 | <- 3
5 -> | 6 | 0 | <- 2
6 -> |_5_|_5_| <- 1
         1         
Turn: P2
P1               P2
     ____0____     
1 

In [6]:
import random

def simple_simulation_100_games():
    random.seed(35) 

    results = []  

    for _ in range(100):
        game = Mancala()
        while not game.winning_eval():
            if game.current_player == 1:
                valid_pits = [i + 1 for i in range(game.pits_per_player) if game.board[i] > 0]
            else:
                valid_pits = [i + 1 for i in range(game.pits_per_player) 
                              if game.board[i + game.pits_per_player + 1] > 0]
            
            if not valid_pits:
                break  
            
            chosen_pit = random.choice(valid_pits)
            game.play(chosen_pit)
        
        if game.board[game.p1_mancala_index] > game.board[game.p2_mancala_index]:
            results.append(1)  
        elif game.board[game.p1_mancala_index] < game.board[game.p2_mancala_index]:
            results.append(2) 
        else:
            results.append(0)  

    p1_wins = results.count(1)
    p2_wins = results.count(2)
    ties = results.count(0)
    
    p1_win_pct = (p1_wins / 100) * 100
    p2_win_pct = (p2_wins / 100) * 100
    tie_pct = (ties / 100) * 100

    print(f"Player 1 wins: {p1_wins} ({p1_win_pct:.1f}%)")
    print(f"Player 2 wins: {p2_wins} ({p2_win_pct:.1f}%)")
    print(f"Ties: {ties} ({tie_pct:.1f}%)")

# Run the simulation
simple_simulation_100_games()



Player 1 wins!
Player 2 wins!
Player 2 wins!
It's a tie!
Player 2 wins!
Player 2 wins!
Player 1 wins!
Player 2 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 2 wins!
Player 2 wins!
Player 1 wins!
Player 1 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 1 wins!
Player 1 wins!
Player 2 wins!
Player 2 wins!
Player 1 wins!
Player 2 wins!
It's a tie!
Player 1 wins!
It's a tie!
Player 1 wins!
Player 2 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 2 wins!
It's a tie!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 1 wins!
Player 2 wins!
Player 1 wins!
Player 1 wins!
Player 2 wins!
Player 1 wins!
Player 1 wins!
Player 2 wins!
Player 2 wins!
Player 1 wins!
Player 1 wins!
Player 2 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 1 wins!
Player 2 wins!
Player 2 wins!
Player 1 wins!
Player 1 wins!
Player 